In [1]:
import sys  
sys.path.insert(0, '')
import searchandinvoke as sandi
#persistant the best_model in the disc but described as triple knowledge 
import joblib
from joblib import dump,load
from rdflib import Graph, Literal, RDF, URIRef
# rdflib knows about quite a few popular namespaces, like W3C ontologies, schema.org etc.
from rdflib.namespace import FOAF , XSD
from rdflib.namespace import NamespaceManager
from rdflib import BNode
from uuid import uuid4
#from inspect import getmembers, isfunction
#print(getmembers(sandi, isfunction))
import pickle
import joblib
from joblib import dump,load

# task implemenation plan
1. createTask function: taskName, generate a taskId, input datatype, output datatype, aim description

    1.1 create a knowledge space (RDF) with initialing a graph to set name and id triples
    
    1.2 based on pipeline knowledge or input and output parameters semantic matching to select next microservice
    
    1.3 invoke the service to get output
    
    1.4 based on the output to generation knowledge into knolwedge space - knowledge generation function
    
    1.5 repeat 1.2 until final output request satisfied.
    
    
2. knowledge generatie fucntion

# pipeline knoweldge generation py
    

In [2]:
output_memery={}

In [3]:
def OperatingTask (task_name,input_file_path,desir_output_type, task_description,namespace):
    g = Graph()
    g.parse("tasklog.n3")
    if len(task_name) > 0 and len(desir_output_type)> 0:
        if (namespace==''):
            namespace='http://aimicroservice.derby.ac.uk'
        task_id = str(uuid4())
        task= URIRef(namespace+'/'+task_id)
        #verb:
        _type = URIRef(namespace+'/task')
        has_input = URIRef(namespace+'/input')
        has_output = URIRef(namespace+'/output')
        has_ioct = URIRef(namespace+'/iocategory')
        has_iodt = URIRef(namespace+'/iodatatype')
        has_ioshape = URIRef(namespace+'/ioshape')
        has_description = URIRef(namespace+'/description')
        #object:
        g.add((task, RDF.type, _type))
        if (len(task_description)>0): 
            g.add((task, has_description, Literal(task_description,lang="en")))     
        if len(input_file_path) > 0: 
            _input = BNode()
            input_type = input_file_path.split('.')[1]
            g.add((task, has_input, _input))
            g.add((_input, has_iodt, URIRef(namespace+'/'+input_type)))        
            _output = BNode()
        g.add((task, has_output, _output)) 
        for x in desir_output_type: 
            #print(len(x.split('.')))
            if len(x.split('.'))==1:
                #print(x)
                g.add((_output, has_iodt, URIRef(namespace+'/'+x)))
            if len(x.split('.'))>1:
                g.add((_output, has_ioct, URIRef(namespace+'/'+x.split('.')[0])))
                g.add((_output, has_iodt, URIRef(namespace+'/'+x.split('.')[1])))
            if len(x.split('.'))>2:
                g.add((_output, has_ioshape, Literal(x.split('.')[2],lang="en")))
        print(task_id+' task is generated')
        output_value=composition_flow(input_file_path,desir_output_type,input_file_path, task_id)
        g.serialize(destination='tasklog'+".n3")
        return output_value
    else:
        return 'erro: nothing is generated'
        

In [4]:
def composition_flow(input_path,desir_output,input_data, task_id):
    servicelist = []
    
    print('############Composition start##############')
    output_value = sandi.searchandinvoke ('',[input_path.split('.')[1]], desir_output, [input_path], task_id, servicelist, output_memery)
    print('out while step: ',output_value[0])
    if type(output_value[0]) is int:
        if output_value[0]==505:
            servicelist.append(output_value[1])    
        input_type_test = [input_path.split('.')[1]]
        input_value = [input_path]
        #type(i) is int
        controlgate=0
        while(type(output_value[0]) is int):
            print('------------In while-----------')
            output_value = sandi.searchandinvoke ('',input_type_test, '', input_value, task_id, servicelist,output_memery)
            #print(output_value[0])
            if type(output_value[0]) is int:
                if (output_value[0]==505 and controlgate<5):
                    servicelist.append(output_value[1])
                    controlgate=controlgate+1
                else:
                    print('---end while and composition fails----')
                    break
            else:
                if output_value[1] not in servicelist:
                    servicelist.append(output_value[1])
                    flag = output_match_nextms(output_value[1])
                    if flag!=0:
                        input_type_test = []
                        input_value = [output_value[0]]
                        #print(input_value)
                        for xf in flag:
                            print('I know here is a problem')
                            input_type_test.append(xf[1]) 
                            print('check input_value: ',len(input_value[0]))
                        output_value = sandi.searchandinvoke ('',input_type_test, desir_output, input_value, task_id, servicelist, output_memery)
                        if (len(output_value)<3):
                            print('something wrong')
                        else:
                            output_memery.update(output_value[2])
                else:
                    print('!!!repeat steps')
                
        print('-------end while-------')
    else:
        servicelist.append(output_value[1])
        output_memery.update(output_value[2])
        print('******** A single service find to complete the task')
    if type(output_value[0]) is int:
        print('############Composition end unsucessfully##############')
    else:
        print('############Composition sucessfully complete##############')
    return output_value

In [5]:
def output_match_nextms(ms_name):
    g = Graph()
    g.parse("registry.n3")
    namespace='http://aimicroservice.derby.ac.uk'
    MService = URIRef(namespace+'/'+ms_name)
    flag=[]
    q = """
        PREFIX ns1: <http://aimicroservice.derby.ac.uk/>
        
        SELECT *
        WHERE {""" 
    if len(ms_name)>0:
        q=q+'ns1:'+ms_name+""" ns1:output ?out . ?out ns1:paramter ?up . {?up ns1:pid ?uid . ?up ns1:iocategory ?c . ?up ns1:iodatatype ?d .} UNION {?up ns1:pid ?uid . ?up ns1:iodatatype ?d}}""" 
        print (q)
        qr = g.query(q)
        flag_memery=[]
        if len(qr)==0:
            print('No output find')
        else:
            for r in qr:
                if r["c"] is None:
                    if r["d"] is None:
                        print ('no output find')
                    else:
                        #print(r["d"].split('/')[-1])
                        if r["d"].split('/')[-1] not in flag_memery:
                            flag.append([r["uid"],r["d"].split('/')[-1]])
                            flag_memery.append(r["d"].split('/')[-1])
                else:
                    #print(r["c"].split('/')[-1])
                    cate=r["c"].split('/')[-1]
                    if r["d"] is None:
                        #print ('no specific output')
                        if cate not in flag_memery:
                            flag.append([r["uid"],cate])
                            flag_memery.append(cate)
                    else:
                        #print(r["d"].split('/')[-1])
                        if r["d"].split('/')[-1] not in flag_memery:
                            flag.append([r["uid"],cate+'.'+r["d"].split('/')[-1]])
                            flag_memery.append(r["d"].split('/')[-1])
        print(flag)
        return flag
    else:
        return 0

In [6]:
value = OperatingTask ('test','heart.csv',['pipeline'],'testing','')

abfff3fb-91f1-4246-8856-5cacb844aa93 task is generated
############Composition start##############
No microservice has been find
out while step:  404
------------In while-----------
this is service found loadnormalpddata and previous output_memery: {}
has_input_number:  1 invoke_required_input_number 1
datafile []
pandas ['datafile']
output:::: ['datafile', 'pandas']
Successfully imported loadnormalpddata microservice.
this is semantic!!!! ['datafile', 'pandas']
Details data knolwedge has been saved in ProfileReport
this is semantic!!!! ['datafile', 'pandas']

        PREFIX ns1: <http://aimicroservice.derby.ac.uk/>
        
        SELECT *
        WHERE {ns1:loadnormalpddata ns1:output ?out . ?out ns1:paramter ?up . {?up ns1:pid ?uid . ?up ns1:iocategory ?c . ?up ns1:iodatatype ?d .} UNION {?up ns1:pid ?uid . ?up ns1:iodatatype ?d}}
[[rdflib.term.Literal('0', lang='en'), 'datafile.pandas']]
I know here is a problem
check input_value:  303
No microservice has been find
something wrong

#y_train# ['datafile', 'split']
this is the value in flag: ['datafile', 'split', '#y_train#'] 3
this is the value in flag_dict: ['datafile', 'split', '#y_train#']
this is the value in flag_dict: ['model', 'dict']
#X_train# ['datafile', 'split']
this is the value in flag: ['datafile', 'split', '#X_train#'] 3
this is the value in flag_dict: ['datafile', 'split', '#X_train#']
last service name and input spliting {'2': ['datafile', 'split', '#y_train#'], '0': ['model', 'dict'], '1': ['datafile', 'split', '#X_train#']}
datafile []
split ['datafile']
#X_train, X_test, y_train, y_test# ['datafile', 'split']
output:::: ['datafile', 'split', '#X_train, X_test, y_train, y_test#']
find_match_paramenter---
['datafile', 'split', '#y_train#']
output=3: X_train input=1: 2 = y_train
output=3:  X_test input=1: 2 = y_train
output=3:  y_train input=1: 2 = y_train
pair created
output=3:  y_test input=1: 2 = y_train
['model', 'dict']
dict datafile
['datafile', 'split', '#X_train#']
output=3: X_train input=

datafile []
split ['datafile']
#X_train, X_test, y_train, y_test# ['datafile', 'split']
output:::: ['datafile', 'split', '#X_train, X_test, y_train, y_test#']
find_match_paramenter---
['datafile', 'split', '#X_train#']
output=3: X_train input=1: 0 = X_train
pair created
output=3:  X_test input=1: 0 = X_train
output=3:  y_train input=1: 0 = X_train
output=3:  y_test input=1: 0 = X_train
important! 1 3
Successfully imported pipelinemodels microservice.
assign matched output parameter:  0  -->input
Details data knolwedge has been saved in ProfileReport
this is semantic!!!! ['datafile', 'split', '#X_train, X_test, y_train, y_test#']

        PREFIX ns1: <http://aimicroservice.derby.ac.uk/>
        
        SELECT *
        WHERE {ns1:pipelinemodels ns1:output ?out . ?out ns1:paramter ?up . {?up ns1:pid ?uid . ?up ns1:iocategory ?c . ?up ns1:iodatatype ?d .} UNION {?up ns1:pid ?uid . ?up ns1:iodatatype ?d}}
[[rdflib.term.Literal('0', lang='en'), 'model.dict']]
I know here is a problem
check

end combination and invoke:--------------
-------end while-------
############Composition sucessfully complete##############


In [24]:
value[0][1]

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(add_indicator=True,
                                                                                 strategy='median')),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000025F71CD0C10>),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='constant')),
                                              

In [35]:
value[2]['datafile.split.#X_train, X_test, y_train, y_test#'][0]['8']

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(add_indicator=True,
                                                                                 strategy='median')),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000025F71CD0C10>),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='constant')),
                                              

In [30]:
value[2]

{'datafile.pandas':      age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  \
 0     63    1   3       145   233    1        0      150      0      2.3   
 1     37    1   2       130   250    0        1      187      0      3.5   
 2     41    0   1       130   204    0        0      172      0      1.4   
 3     56    1   1       120   236    0        1      178      0      0.8   
 4     57    0   0       120   354    0        1      163      1      0.6   
 ..   ...  ...  ..       ...   ...  ...      ...      ...    ...      ...   
 298   57    0   0       140   241    0        1      123      1      0.2   
 299   45    1   3       110   264    0        1      132      0      1.2   
 300   68    1   0       144   193    1        1      141      0      3.4   
 301   57    1   0       130   131    0        1      115      1      1.2   
 302   57    0   1       130   236    0        0      174      0      0.0   
 
      slope  ca  thal  target  
 0        0   0     1  

In [2]:
output_value = sandi.searchandinvoke ('ETL_load',['datafile.csv'], ['datafile.pandas'], ['heart.csv'], 'task1')


        PREFIX ns1: <http://aimicroservice.derby.ac.uk/>
        
        SELECT *
        WHERE {?ms rdf:type ns1:ETL_load .
     ?ms ns1:input ?in . ?in ns1:paramter ?ip .?ip ns1:iocategory ns1:datafile . ?ip ns1:iodatatype ns1:csv . ?ms ns1:output ?out . ?out ns1:paramter ?up .?up ns1:iocategory ns1:datafile . ?up ns1:iodatatype ns1:pandas .}
loadnormalpddata
Successfully imported loadnormalpddata microservice.
input:  heart.csv ->knowledge:  ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target']


In [5]:
output_value[1]

'loadnormalpddata'

In [4]:
output_value1 = sandi.searchandinvoke ('ETL_transform','', ['pipeline.processor'], '','task1')

scalling
Successfully imported scalling microservice.


In [5]:
output_value2 = sandi.searchandinvoke ('DE_split',['datafile.pandas','varable.str'], ['datafile.pandas'], [output_value,'target'],'task1')

spliting
Successfully imported spliting microservice.


In [6]:
output_value3 = sandi.searchandinvoke ('MLmodel_classification','', ['model.logistic_regression'], '','task1')

lrsklmodel
Successfully imported lrsklmodel microservice.


In [8]:
output_value4 = sandi.searchandinvoke ('ML_featuretest',['datafile.logistic_regression','model.pandas','pipeline.processor'], ['model.dict'], [output_value2[0],output_value2[1],output_value3,output_value1],'task1')

featureengineering
Successfully imported featureengineering microservice.


In [10]:
output_value5 = sandi.searchandinvoke ('ML_optimisation',['datafile.pandas','model.dict'], ['score.float','model.dict'], [output_value4,output_value2[3],output_value2[5]],'task1')

featuremodelevaluation
Successfully imported featuremodelevaluation microservice.


In [12]:
output_value5[1]

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(add_indicator=True,
                                                                                 strategy='median')),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000023783CC3520>),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='constant')),
                                              

In [16]:
select_pipem=output_value5[1]
select_pipem.fit(output_value2[0],output_value2[1])
print(select_pipem.named_steps['rfe'].support_,select_pipem.named_steps['rfe'].ranking_)

[False  True  True False False False False False False  True False  True
 False] [ 9  1  1  6  8 10  7  2  4  1  5  1  3]


In [18]:
output_value.columns.values.tolist()[1]

'sex'